In [1]:
import numpy as np
import transformers
from transformers import Trainer
transformers.logging.set_verbosity_error()
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset, load_metric
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [2]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

def read_and_tokenize_data(train,test,evaluate):
    d = load_dataset('pandas', data_files={"train":train, "test":test,"evaluate":evaluate})
    for i in d:
        d[i]= d[i].remove_columns(column_names = ['__index_level_0__'])
    d = d.map(tokenize_function, batched=True)
    return d

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Reading data

In [3]:
data_dict = read_and_tokenize_data("data/authors_train.pkl",\
                               "data/authors_test.pkl",\
                                   "data/authors_validation.pkl")

data_dict_2 = read_and_tokenize_data("data/imdb_train.pkl",\
                               "data/imdb_test.pkl",\
                                   "data/imdb_validation.pkl")

Using custom data configuration default-571dfb15c0701b97
Reusing dataset pandas (/scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-571dfb15c0701b97/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-571dfb15c0701b97/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-189e360544a47ec7.arrow
Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-571dfb15c0701b97/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-0673ad7c77c63b62.arrow
Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-571dfb15c0701b97/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-4887fa858c0185a8.arrow
Using custom data configuration default-f45029a52432aaca
Reusing dataset pandas (/scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-f45029a52432aaca/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-f45029a52432aaca/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-7244438eaf82d0b2.arrow
Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-f45029a52432aaca/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-11f200e5c63e1413.arrow
Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-f45029a52432aaca/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-0d9f295005f739d6.arrow


#### Training model

In [4]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 4)

model_2 = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = 2)

In [5]:
training_args = TrainingArguments(
    output_dir='results',
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps = 250,
    save_steps = 250,
    load_best_model_at_end = True
)


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args = training_args,
    train_dataset=data_dict["train"],
    eval_dataset=data_dict["evaluate"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


trainer_2 = Trainer(
    model=model_2,
    args = training_args,
    train_dataset=data_dict_2["train"],
    eval_dataset=data_dict_2["evaluate"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [6]:
trainer_2.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 2700
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6760
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to results/checkpoint-250
Configuration saved in results/checkpoint-250/config.json


{'eval_loss': 0.42121168971061707, 'eval_accuracy': 0.88, 'eval_runtime': 1.4986, 'eval_samples_per_second': 100.092, 'eval_steps_per_second': 12.678, 'epoch': 0.74}


Model weights saved in results/checkpoint-250/pytorch_model.bin
tokenizer config file saved in results/checkpoint-250/tokenizer_config.json
Special tokens file saved in results/checkpoint-250/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8


{'loss': 0.3439, 'learning_rate': 1.85207100591716e-05, 'epoch': 1.48}


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json


...

Model weights saved in results/checkpoint-6500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-6500/tokenizer_config.json
Special tokens file saved in results/checkpoint-6500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 150
  Batch size = 8
Saving model checkpoint to results/checkpoint-6750
Configuration saved in results/checkpoint-6750/config.json


{'eval_loss': 0.8569173216819763, 'eval_accuracy': 0.92, 'eval_runtime': 1.4369, 'eval_samples_per_second': 104.39, 'eval_steps_per_second': 13.223, 'epoch': 19.97}


Model weights saved in results/checkpoint-6750/pytorch_model.bin
tokenizer config file saved in results/checkpoint-6750/tokenizer_config.json
Special tokens file saved in results/checkpoint-6750/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1500 (score: 0.394641250371933).


{'train_runtime': 1842.6312, 'train_samples_per_second': 29.306, 'train_steps_per_second': 3.669, 'train_loss': 0.05029036337585525, 'epoch': 20.0}


TrainOutput(global_step=6760, training_loss=0.05029036337585525, metrics={'train_runtime': 1842.6312, 'train_samples_per_second': 29.306, 'train_steps_per_second': 3.669, 'train_loss': 0.05029036337585525, 'epoch': 20.0})

#### Prediction and metrics

In [44]:
def make_predictions(trainer,test):
    predictions = trainer.predict(test)
    y_true = test["label"]
    y_pred = np.argmax(predictions.predictions, axis=-1)
    confusion_matrix(y_true, y_pred)
    metric = load_metric("accuracy")
    metric.compute(predictions=y_pred, references=y_true)

#### Saving model

In [7]:
def save_model(trainer,tokenizer,trainer_path,tokenizer_path):
    trainer.save_model(trainer_path)
    tokenizer.save_pretrained(tokenizer_path)
    
save_model(trainer,tokenizer,"models/author_classification/model","models/author_classification/tokenizer")
save_model(trainer_2,tokenizer,"models/sentiment_classification/model","models/sentiment_classification/tokenizer")

Saving model checkpoint to models/sentiment_classification/model
Configuration saved in models/sentiment_classification/model/config.json
Model weights saved in models/sentiment_classification/model/pytorch_model.bin
tokenizer config file saved in models/sentiment_classification/model/tokenizer_config.json
Special tokens file saved in models/sentiment_classification/model/special_tokens_map.json
tokenizer config file saved in models/sentiment_classification/tokenizer/tokenizer_config.json
Special tokens file saved in models/sentiment_classification/tokenizer/special_tokens_map.json


#### Loading this model

In [12]:
def load_model(model_path,tokenizer_path,num_labels):
    model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = num_labels)
    tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)
    trainer = Trainer(model = model)
    return (model, trainer, tokenizer)
    

model, trainer, tokenizer = load_model("models/author_classification/model","models/author_classification/tokenizer",4)

model_2, trainer_2, _ = load_model("models/sentiment_classification/model","models/sentiment_classification/tokenizer",2)

loading configuration file models/author_classification/model/config.json
Model config RobertaConfig {
  "_name_or_path": "models/author_classification/model",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size"

In [13]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def read_and_tokenize_data(data):
    d = load_dataset('pandas', data_files=data,split="train")
    d = d.map(tokenize_function, batched=True)
    return d


def make_predictions(trainer,data_path):
    test_data = read_and_tokenize_data(data_path)
    predictions = trainer.predict(test_data)
    y_true = test_data["label"]
    y_pred = np.argmax(predictions.predictions, axis=-1)
    print(confusion_matrix(test_data["label"], y_pred))
    metric = load_metric("accuracy")
    print(metric.compute(predictions=y_pred, references=y_true))



make_predictions(trainer,"data/authors_test.pkl")

make_predictions(trainer_2,"data/imdb_test.pkl")

Using custom data configuration default-79c54807d2b16118
Reusing dataset pandas (/scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-79c54807d2b16118/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)
Loading cached processed dataset at /scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-79c54807d2b16118/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade/cache-68ac3e80a3ae88ac.arrow
The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Prediction *****
  Num examples = 149
  Batch size = 8


[[48  0  0  0]
 [ 0 21  0  0]
 [ 1  0  5  0]
 [ 2  1  0 71]]
{'accuracy': 0.9731543624161074}


Using custom data configuration default-34266c77c54981d3
Reusing dataset pandas (/scratch/lustre/home/doma6660/.cache/huggingface/datasets/pandas/default-34266c77c54981d3/0.0.0/6197c1e855b639d75a767140856841a562b7a71d129104973fe1962594877ade)


  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Prediction *****
  Num examples = 150
  Batch size = 8


[[73  4]
 [ 9 64]]
{'accuracy': 0.9133333333333333}


In [5]:
from transformers import TextClassificationPipeline

In [20]:
pipeline = TextClassificationPipeline(model=model,tokenizer=tokenizer)

pipeline("Classify this!")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)

In [21]:
pipeline_2 = TextClassificationPipeline(model=model_2,tokenizer=tokenizer)

pipeline_2("Classify this!")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper__index_select)